# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [7]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [8]:
itemset = pd.read_csv("BlackFriday.csv").dropna().drop_duplicates()
itemset

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
5,1000013,P00350442,M,46-50,1,C,3,1,2,3.0,15.0
6,1000013,P00155442,M,46-50,1,C,3,1,1,11.0,15.0
7,1000013,P0094542,M,46-50,1,C,3,1,2,4.0,9.0
8,1000015,P00161842,M,26-35,7,A,1,0,10,13.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
233584,1006023,P00262242,M,26-35,0,C,2,1,1,11.0,16.0
233586,1006026,P00110742,M,36-45,6,C,1,1,1,2.0,8.0
233588,1006030,P00129842,M,26-35,17,C,1,1,6,8.0,15.0
233591,1006033,P00127642,M,51-55,13,B,1,1,1,2.0,15.0


#### Pregunta Nº1: Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto. (1 punto)


In [9]:
# Esto no sirve, pero dejar por si acaso (antes de entregar se borra)

# transaccionesProduct = itemset[['User_ID', 'Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']].groupby('User_ID').agg(lambda x: list(x)).reset_index()
# transaccionesUser = itemset[['User_ID', 'Gender', 'Age']].groupby('User_ID').agg(lambda x: "".join(set(x))).reset_index()

# transaccionesUser[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']] = transaccionesProduct[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
# transacciones = transaccionesUser[['User_ID', 'Gender', 'Age', 'Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]

# transacciones

# tablaProductos = itemset[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']].drop_duplicates()
# tablaProductos

Configuraciones de sopoorte y frecuencia para género

In [10]:
tablaGenero = itemset[['Product_ID', 'Gender']].groupby('Gender').agg(lambda x: list(x)).reset_index()
tablaGenero

,Gender,Product_ID
0,F,"[P00053842, P00159542, P00346442, P00177442, P..."
1,M,"[P00350442, P00155442, P0094542, P00161842, P0..."


- Para k = 1

In [11]:
te = TransactionEncoder()
te_ary = te.fit(tablaGenero['Product_ID'].to_list()).transform(tablaGenero['Product_ID'].to_list())
df = pd.DataFrame(te_ary, columns=te.columns_)
df


frequent_itemsets_genero = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets_genero

In [ ]:
tablaEdad = itemset[['Product_ID', 'Age']].groupby('Age').agg(lambda x: list(x)).reset_index()
tablaEdad

,Age,Product_ID
0,0-17,"[P00015542, P00000142, P00251942, P00053842, P..."
1,18-25,"[P00216342, P00112642, P00171542, P00148642, P..."
2,26-35,"[P00053842, P00161842, P00046742, P00196542, P..."
3,36-45,"[P00000142, P00117842, P00110842, P00243942, P..."
4,46-50,"[P00350442, P00155442, P0094542, P00344442, P0..."
5,51-55,"[P00177442, P00359242, P00324442, P00085242, P..."
6,55+,"[P00112142, P00062842, P00295942, P00235942, P..."


In [ ]:
tablaTProd1 = itemset[['Product_ID', 'Product_Category_1']].groupby('Product_Category_1').agg(lambda x: list(x)).reset_index()
tablaTProd1

,Product_Category_1,Product_ID
0,1,"[P00155442, P00046742, P00059642, P00030842, P..."
1,2,"[P00350442, P0094542, P0096642, P00171542, P00..."
2,3,"[P00000142, P00216342, P0095842, P00190142, P0..."
3,4,"[P00053842, P00183642, P00003442, P00135242, P..."
4,5,"[P00196542, P00344442, P00117842, P00034742, P..."
5,6,"[P00148642, P00129842, P00310842, P00346042, P..."
6,8,"[P00086442, P00154742, P00153842, P00085242, P..."
7,10,"[P00161842, P00159542, P00159542, P00116142, P..."
8,11,"[P00214642, P00324442, P00324442, P00341942, P..."
9,12,"[P00273442, P00273442, P00273442, P00273442, P..."


In [ ]:
tablaTProd2 = itemset[['Product_ID', 'Product_Category_2']].groupby('Product_Category_2').agg(lambda x: list(x)).reset_index()
tablaTProd2

,Product_Category_2,Product_ID
0,2.0,"[P00046742, P00059642, P00030842, P00346442, P..."
1,3.0,"[P00350442, P0096642, P00277642, P00277642, P0..."
2,4.0,"[P0094542, P00000142, P00216342, P0095842, P00..."
3,5.0,"[P00053842, P00171542, P00359242, P00295942, P..."
4,6.0,"[P00177442, P00177442, P00201442, P00201442, P..."
5,8.0,"[P00196542, P00344442, P00223542, P00165642, P..."
6,9.0,"[P00074642, P00192042, P00074642, P00324742, P..."
7,10.0,"[P00148642, P00067842, P00148642, P00116942, P..."
8,11.0,"[P00155442, P00262242, P00262242, P00209742, P..."
9,12.0,"[P00128242, P00120742, P00088542, P00128242, P..."


In [ ]:
tablaTProd3 = itemset[['Product_ID', 'Product_Category_3']].groupby('Product_Category_3').agg(lambda x: list(x)).reset_index()
tablaTProd3

,Product_Category_3,Product_ID
0,3.0,"[P00059642, P00059642, P00059642, P00059642, P..."
1,4.0,"[P0096642, P00251642, P00144042, P00144042, P0..."
2,5.0,"[P00110842, P00000142, P00216342, P00112642, P..."
3,6.0,"[P00042142, P00182142, P00173942, P00143642, P..."
4,8.0,"[P00177442, P00177442, P00201442, P00110742, P..."
5,9.0,"[P0094542, P0094542, P00242742, P00000242, P00..."
6,10.0,"[P00310842, P00277642, P00277642, P00109242, P..."
7,11.0,"[P00198742, P00241642, P00217542, P00241642, P..."
8,12.0,"[P00053842, P0095842, P00004842, P00113142, P0..."
9,13.0,"[P00148642, P00145942, P00015542, P00062842, P..."


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transacciones['Product_ID'].to_list()).transform(transacciones['Product_ID'].to_list())
df = pd.DataFrame(te_ary, columns=te.columns_)
df


frequent_itemsets = apriori(df, min_support=0.003, use_colnames=True)
# K=1 -> 137 items
# K=2 -> 
frequent_itemsets


MemoryError: Unable to allocate 31.7 GiB for an array with shape (2012302, 3, 5645) and data type bool

#### Pregunta Nº2: ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos (1 punto)

#### Pregunta Nº3: ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto) consideradas? ¿Por qué? (1 punto)

#### Pregunta Nº4: Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y min_threshold. Justifica los resultados. (1 punto)


#### Pregunta Nº5: Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)